In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
# 설치,  from, import
!pip install flask-ngrok > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1

# with open('drive/MyDrive/project/static/ngrok_auth.txt') as nf:
#     ngrok_auth = nf.read()
# !cat drive/MyDrive/project/static/ngrok_auth.txt 

!ngrok authtoken $ngrok_auth
!pip install efficientnet_pytorch

The system cannot find the path specified.
The system cannot find the path specified.
'ngrok' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# 웹- 업로드 시 실행하지 않게 미리 임폴트
import torchvision
from torchvision import transforms
import os
from torch.utils.data import Dataset,DataLoader
import torch

In [4]:
# 웹페이지에서 이미지를 업로드 받은 후 연산시간을 줄이기 위해 미리 로드하는 것

# PATH
PATH1 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model1.pt'  # 모델1
PATH2 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model2.pt'  # 모델2
PATH3 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model3.pt'  # 모델3
PATH4 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model4.pt'  # 모델4
PATH5 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model5.pt'  # 모델5
PATH6 = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model6.pt'  # 모델6

# cuda        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model load        
model1 = torch.load(PATH1, map_location=device)
model2 = torch.load(PATH2, map_location=device)
model3 = torch.load(PATH3, map_location=device)
model4 = torch.load(PATH4, map_location=device)
model5 = torch.load(PATH5, map_location=device)
model6 = torch.load(PATH6, map_location=device)

# 아웃풋, 로스, 프레딕, 아큐러시
# 모델 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
model1.eval() 
model2.eval()
model3.eval()
model4.eval()
model5.eval()
model6.eval()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/scalp_weights/aram_model1.pt'

In [ ]:
## Web Server Code
app = Flask(__name__, static_folder='/content/drive/MyDrive/project/static/upload', # 이곳에 유저가 업로드한 파일을 save
                      template_folder='/content/drive/MyDrive/project/templates')
run_with_ngrok(app)

@app.route('/')
def home():
    menu = {'home':1, 'menu':0}
    return render_template('index.html', menu=menu)

@app.route('/menu', methods=['GET','POST'])
def menu():
    menu = {'home':0, 'menu':1}
    if request.method == 'GET':
        languages = [
            #{'disp':'영어', 'val':'en'},
            #{'disp':'일어', 'val':'jp'},
            #{'disp':'중국어', 'val':'cn'},
            #{'disp':'프랑스어', 'val':'fr'},
            #{'disp':'스페인어', 'val':'es'}
        ]
        
        #업로드폴더 리셋
        #upload_file_path = "/content/drive/MyDrive/project/static/upload/upload/upload.jpg"
        #if os.path.exists(upload_file_path):
        #    os.remove(upload_file_path) # 업로드폴더에 파일을삭제

        return render_template('menu.html', menu=menu,
                                options=languages)   # 서버에서 클라이언트로 정보 전달
    else:
        # 사용자가 입력한 정보를 서버가 읽음
        # index = request.form['index']
        # lang = request.form['lang']
        # lyrics = request.form['lyrics']
        #print(lang, '\n', index, '\n', lyrics, sep='')
        # 사용자가 입력한 파일을 읽어서 upload 디렉토리에 저장
        f_image = request.files['image']
        fname = f_image.filename                # 사용자가 입력한 파일 이름
        newname = 'upload.jpg' # 업로드폴더를리셋하기위해일정한새이름을지정 upload.jpg로 업로드
        filename = os.path.join(app.static_folder, 'upload/') + newname  # 유저가 업로드한 사진이 저장되는 공간과 파일이름 
                                        # static_folder/upload 가 경로 파일네임 : fname 유저가 업로드한 파일의 이름 
        f_image.save(filename) #파일세이브
       
###########################################################################################################################################

## model.test code

# test 이미지파일 전처리, 텐서화
        # 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
        transforms_test = transforms.Compose([     
                                                transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                            ])
        # root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
        testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/static/upload' ,
                            transform = transforms_test)
      
# DataLoader를 통해 네트워크에 올리기 
        testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

        # correct = 0
        
        # 로스 연산
        # import torch.nn.functional as F   # F : 테스트_로스 연산 함수
        # test_loss = 0
        
        from tqdm import tqdm # 진행률 표시를 위한

        if __name__ == '__main__':
            with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
                    for data, target in tqdm(testloader):                                   
                        data, target  = data.to(device), target.to(device) 
                        
                        output1 = model1(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                        output2 = model2(data) 
                        output3 = model3(data) 
                        output4 = model4(data) 
                        output5 = model5(data) 
                        output6 = model6(data)
                 
# predict # # 0~3값만 뽑기 
        m1p = output1.argmax(dim=1, keepdim=True)[0][0].tolist()
        m2p = output2.argmax(dim=1, keepdim=True)[0][0].tolist()
        m3p = output3.argmax(dim=1, keepdim=True)[0][0].tolist()
        m4p = output4.argmax(dim=1, keepdim=True)[0][0].tolist()
        m5p = output5.argmax(dim=1, keepdim=True)[0][0].tolist()
        m6p = output6.argmax(dim=1, keepdim=True)[0][0].tolist()
 
# 진단
        d_list = [] # 두피유형진단

        # 두피 유형 판단
        if m1p == 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d1 = '정상입니다.'
            d_list.append(d1)
        elif m1p != 0 and m2p == 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d2 = '건성 두피입니다.'
            d_list.append(d2)
        elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d3 = '지성 두피입니다.'
            d_list.append(d3)
        elif m2p == 0 and m3p != 0 and m4p == 0 and m5p == 0 and m6p == 0 :
            d4 = '민감성 두피입니다.'
            d_list.append(d4)
        elif m2p != 0 and m3p != 0 and m4p == 0 and m6p == 0 :
            d5 = '지루성 두피입니다.'
            d_list.append(d5)
        elif m3p == 0 and m4p != 0 and m6p == 0 :
            d6 = '염증성 두피입니다.'
            d_list.append(d6)
        elif m3p == 0 and m4p == 0 and m5p != 0 and m6p == 0 :
            d7 = '비듬성 두피입니다.'
            d_list.append(d7)
        elif m1p == 0 and m2p != 0 and m3p == 0 and m4p == 0 and m5p == 0 and m6p != 0 :
            d8 = '탈모입니다.'
            d_list.append(d8)
        else:
            d9 = '복합성 두피입니다.'
            d_list.append(d9)

########################################################################################################################################
 
## Web Server Code

 # 모델 실행후 결과를 돌려줌
        final = d_list[0] # 두피유형판단
        result = {'미세각질':m1p, '피지과다':m2p,'모낭사이홍반':m3p,'모낭홍반농포':m4p,'비듬':m5p,'탈모':m6p}
        final2 = '0:양호, 1:경증, 2:중등도, 3:중증' 
        mtime = int(os.stat(filename).st_mtime) # 업로드한 시간값불러오기 > 큐변경 > 화면갱신 

        #os.remove("/content/drive/MyDrive/project/static/upload/upload/4.jpg") # 업로드폴더에업로드파일을삭제
        

        return render_template('menu_res.html',  final2=final2,final=final, result=result, menu=menu,
                                fname=newname, mtime=mtime)
            
if __name__ == '__main__':
    app.run()

# 코드간략

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d31c-34-70-115-176.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:12:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:12:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:12:55] "GET /menu HTTP/1.1" 200 -
100%|██████████| 1/1 [00:06<00:00,  6.97s/it]
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:13:09] "POST /menu HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:13:09] "GET /upload/upload/upload.jpg?q=1661299982 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:17:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:17:28] "GET /menu HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  1.36it/s]
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:17:35] "POST /menu HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:17:35] "GET /upload/upload/upload.jpg?q=1661300254 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Aug/2022 00:17:53] "GET /menu HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  1.33it/s]
INFO:werkzeug:1